In [1]:
from keras.layers import Bidirectional , GRU ,LSTM
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

from keras.layers import Embedding

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.models import Sequential

import numpy as np
from keras import backend as K

Using Theano backend.
C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\hp\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from keras.layers import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
import theano.ifelse
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from sklearn.model_selection import StratifiedKFold

## loading the data

In [3]:
train_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/train_medical.csv')
test_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/test_medical.csv')
validation_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/validation_medical.csv')

In [4]:
columns_to_drop = ['_unit_id', '_created_at', '_canary', '_id', '_started_at', '_channel',
       '_trust', '_worker_id', '_country', '_region', '_city', '_ip',
       'direction', 'b1', 'b2', 'direction_gold', 'e1', 'e2',
       'relex_relcos', 'sent_id', 'twrex', 'term1' , 'term2']


train_data = train_data.drop( columns_to_drop , axis = 1)
test_data = test_data.drop(columns_to_drop , axis = 1)
validation_data = validation_data.drop( columns_to_drop , axis = 1)

train_data.loc[train_data.relation == 'is location of' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'diagnosed by' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'contraindicates' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'location' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'location of' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'is diagnosed by' , 'relation'] = 'Others'


test_data.loc[test_data.relation == 'is location of' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'diagnosed by' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'contraindicates' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'location' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'location of' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'is diagnosed by' , 'relation'] = 'Others'

#columns_to_drop = ['_unit_id', '_created_at', '_canary', '_id', '_started_at', '_channel',
      # '_trust', '_worker_id', '_country', '_region', '_city', '_ip',
       #'direction', 'b1', 'b2', 'direction_gold', 'e1', 'e2',
       #'relex_relcos', 'sent_id', 'twrex', 'term1' , 'term2']


#validation_data = validation_data.drop( columns_to_drop , axis = 1)

validation_data.loc[validation_data.relation == 'is location of' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'diagnosed by' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'contraindicates' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'location' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'location of' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'is diagnosed by' , 'relation'] = 'Others'

In [5]:
print(train_data.shape , test_data.shape , validation_data.shape)

(13340, 2) (4566, 2) (4270, 2)


## Preprocessing with keras

In [6]:
train_data_list = [ x for x in train_data['sentence']]
test_data_list = [ x for x in test_data['sentence']]
validation_data_list = [ x for x in validation_data['sentence']]

In [7]:
## Encoding the target value
encoder = LabelEncoder()
encoder.fit(train_data['relation'])
y_train = encoder.transform(train_data['relation'])
y_test = encoder.transform(test_data['relation'])
y_val = encoder.transform(validation_data['relation'])

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
y_val_cat = to_categorical(y_val)


In [8]:
#tokenising 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data_list)

vocab_size = len(tokenizer.word_index) + 1

In [9]:
vocab_size

7982

In [10]:
encoded_docs_train = tokenizer.texts_to_sequences(train_data_list)
encoded_docs_test = tokenizer.texts_to_sequences(test_data_list)
encoded_docs_val = tokenizer.texts_to_sequences(validation_data_list)

In [11]:
# calculate max document length
lengths = [len(s.split()) for s in train_data_list]
max_length = max(lengths)
print(max_length)

97


In [12]:
##########padding
max_length = 97
padded_docs_train = pad_sequences(encoded_docs_train, maxlen = max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen = max_length, padding='post')
padded_docs_val = pad_sequences(encoded_docs_val , maxlen = max_length , padding = 'post')

In [13]:
X_train = padded_docs_train
X_test = padded_docs_test
X_val = padded_docs_val

In [14]:
print(X_train.shape , X_test.shape)

(13340, 97) (4566, 97)


In [15]:
print(y_train.shape , y_test.shape)

(13340,) (4566,)


In [16]:
print(X_val.shape , y_val.shape)

(4270, 97) (4270,)


## word embeddings

In [17]:
filename_pmc = 'C:/Users/hp/Word_embeddings/PubMed-and-PMC-w2v.bin'
model_pmc = gensim.models.KeyedVectors.load_word2vec_format( filename_pmc  , binary=True)

In [18]:
EMBEDDING_DIM_PMC = 200
vocabulary_size =  vocab_size

In [20]:
embedding_matrix_pmc = np.zeros(( vocabulary_size , EMBEDDING_DIM_PMC ))
for word , i in tokenizer.word_index.items():
    try:
        embedding_vector_pmc = model_pmc[word]
    except KeyError:
        embedding_vector_pmc = None
    if embedding_vector_pmc is not None:
        embedding_matrix_pmc[i] = embedding_vector_pmc
        

## model

In [31]:
X = np.concatenate((X_train , X_val ) , axis = 0)

In [32]:
X.shape

(17610, 97)

In [33]:
Y = np.concatenate((y_train , y_val ) , axis = 0)

In [34]:
Y.shape

(17610,)

In [35]:
X_test.shape

(4566, 97)

In [36]:
y_test.shape

(4566,)

In [37]:
kfold = StratifiedKFold( n_splits = 10, shuffle = True)
cvscores = []

for train, test in kfold.split(X, Y):
# create model
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    model = Sequential()
    model.add(Embedding( vocab_size, 200 ,
          weights=[embedding_matrix_pmc], input_length= 97, trainable=False))
    model.add(SpatialDropout1D(0.3))
    model.add(Bidirectional(GRU( 100 ,return_sequences= True, dropout=0.3, recurrent_dropout=0.25)))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(3, activation="softmax"))
   
    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc' , precision , recall])    
    # Fit the model
    y_train_c = to_categorical(Y[train])
    y_val_c   = to_categorical(Y[test])
    model.fit(X[train] , y_train_c , batch_size = 1000 , epochs = 10 , verbose=1 , validation_data = (X[test] ,y_val_c) )
    # evaluate the model
    
    
    scores = model.evaluate(X_test, y_test_cat  , verbose=0)

    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s : %.2f%%" % (model.metrics_names[2] , scores[2]*100 ))
    print("%s : %.2f%%" % (model.metrics_names[3] , scores[3]*100 ))
    precision = scores[2]*100
    recall = scores[3]*100
    ## f1 score
    f1_score = (2 * precision * recall )/( recall + precision )
    print("f1_score")
    print(f1_score)
    cvscores.append(scores[1] * 100)
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 15847 samples, validate on 1763 samples
Epoch 1/10
15847/15847 [==============================] - 70s - loss: 0.9175 - acc: 0.5707 - precision: 0.6320 - recall: 0.3494 - val_loss: 0.7493 - val_acc: 0.7323 - val_precision: 0.7615 - val_recall: 0.6824
Epoch 2/10
15847/15847 [==============================] - 72s - loss: 0.6966 - acc: 0.7362 - precision: 0.7614 - recall: 0.6962 - val_loss: 0.6152 - val_acc: 0.7521 - val_precision: 0.7769 - val_recall: 0.7226
Epoch 3/10
15847/15847 [==============================] - 70s - loss: 0.6200 - acc: 0.7542 - precision: 0.7800 - recall: 0.7150 - val_loss: 0.5820 - val_acc: 0.7703 - val_precision: 0.7988 - val_recall: 0.7192
Epoch 4/10
15847/15847 [==============================] - 73s - loss: 0.5896 - acc: 0.7624 - precision: 0.7921 - recall: 0.7256 - val_loss: 0.5530 - val_acc: 0.7760 - val_precision: 0.8020 - val_recall: 0.7453
Epoch 5/10
15847/15847 [==============================] - 70s - loss: 0.5665 - acc: 0.7733 - precision: 0.8019 

15849/15849 [==============================] - 70s - loss: 0.5239 - acc: 0.7863 - precision: 0.8097 - recall: 0.7535 - val_loss: 0.4601 - val_acc: 0.8154 - val_precision: 0.8311 - val_recall: 0.7899
Epoch 9/10
15849/15849 [==============================] - 70s - loss: 0.5088 - acc: 0.7910 - precision: 0.8165 - recall: 0.7597 - val_loss: 0.4447 - val_acc: 0.8234 - val_precision: 0.8404 - val_recall: 0.7973
Epoch 10/10
15849/15849 [==============================] - 71s - loss: 0.4936 - acc: 0.8035 - precision: 0.8230 - recall: 0.7701 - val_loss: 0.4312 - val_acc: 0.8330 - val_precision: 0.8442 - val_recall: 0.8086
acc: 75.16%
precision : 75.98%
recall : 73.81%
f1_score
74.87571945348529
Train on 15849 samples, validate on 1761 samples
Epoch 1/10
15849/15849 [==============================] - 70s - loss: 0.8614 - acc: 0.6393 - precision: 0.7056 - recall: 0.4135 - val_loss: 0.7104 - val_acc: 0.7411 - val_precision: 0.7766 - val_recall: 0.7149
Epoch 2/10
15849/15849 [=======================

In [38]:
scores = model.evaluate(X_test, y_test_cat)
print("%s : %.2f%%" % (model.metrics_names[1] , scores[1]*100))
print("%s : %.2f%%" % (model.metrics_names[2] , scores[2]*100))
print("%s : %.2f%%" % (model.metrics_names[3] , scores[3]*100))

4544/4566 [============================>.] - ETA: 0sacc : 74.59%
precision : 76.70%
recall : 72.14%


In [39]:
precision = scores[2]*100
recall = scores[3]*100
## f1 score
f1_score = (2 * precision * recall )/( recall + precision )


In [40]:
print(f1_score)

74.35039546663891
